In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

In [2]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [3]:
import sys
!{sys.executable} -m pip install scikeras

In [4]:
import sys
!{sys.executable} -m pip install wandb

In [5]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [6]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='NeuralNet BayesianSearch',job_type='analysis',reinit=True)

wandb: Currently logged in as: sakhawat. Use `wandb login --relogin` to force relogin


In [1]:
import numpy as np

In [2]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [3]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [4]:
from skopt import BayesSearchCV
from scipy.stats import randint, uniform
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

In [11]:
# Define a function to create the neural network model
def create_model(hidden_layer_sizes, activation, solver, alpha):
    model = Sequential()
    for size in hidden_layer_sizes:
        model.add(Dense(size, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [46]:
# Create an instance of the KerasClassifier using the defined model function
keras_classifier = KerasClassifier(model=create_model, optimizer="adam",
    optimizer__learning_rate=0.001,
    epochs=50,
    verbose=0,activation='sigmoid',hidden_layer_sizes=(100, ),alpha=0.0035890947474035823,solver='adamax')

In [47]:
#from skopt.space import Real, Integer, Categorical
# ...
params = {
    'hidden_layer_sizes': [(100,), (150,), (200,), (250,)],
    'activation': ['sigmoid', 'softplus', 'tanh', 'relu'],
    'solver': ['adamax', 'sgd', 'adam'],
    'alpha': (1e-5, 1e-2)
}
# ...

In [48]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [49]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(keras_classifier, params, n_iter=100, cv=cv, n_jobs=-1)

In [50]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

TypeError: 'int' object is not iterable

In [ ]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

In [ ]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [ ]:
# Finish logging with wandb
wandb.finish()

In [6]:
# Create an instance of the KerasClassifier using the defined model function
keras_classifier = KerasClassifier()

In [ ]:
keras_classifier.get